### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [ ]:
import oci
config = oci.config.from_file()

#this is good for pointing to another endpoint than the config file
##ai_client = oci.ai_speech.AIServiceSpeechClient(config,service_endpoint="https://speech.aiservice.us-phoenix-1.oci.oraclecloud.com")

#Using the config file default endpoint
ai_client = oci.ai_speech.AIServiceSpeechClient(config)

In [ ]:
# Transcription job details in these fields
job_display_name = "example_job"
job_compartment_id = "ocid1.compartment.oc1..aaaaaaaafhejmv2sovpb2g7hxjfkonpxzlhb3ued6iutapxbqafjhl6s7xdq" #aispeech Test compartment
job_description = "example with oracle advertising"
bucket_name = "example"
namespace = "axsjzgvicq5h"
new_compartment_id = "ocid1.compartment.oc1..aaaaaaaaw5zoh4xtzfoisabf6qcxtolnwc6vzda3ym6dpamfolmnzvscsd5q" #aispeech LiveLab compartment
new_display_name = "updated_name"
new_description = "new updated description"

#to be used in create_transcription_job
job_model_details = oci.ai_speech.models.TranscriptionModelDetails(domain="GENERIC", language_code="en-US")
job_object_location = oci.ai_speech.models.ObjectLocation(namespace_name=namespace, bucket_name=bucket_name,object_names=["Oracle_Advertising.wav"])
job_input_location = oci.ai_speech.models.ObjectListInlineInputLocation(location_type="OBJECT_LIST_INLINE_INPUT_LOCATION", object_locations=[job_object_location])
job_output_location = oci.ai_speech.models.OutputLocation(namespace_name=namespace, bucket_name=bucket_name, prefix="Speech_Notebook_Demo")


In [ ]:
# Create Transcription Job with details provided
transcription_job_details = oci.ai_speech.models.CreateTranscriptionJobDetails(display_name=job_display_name,
compartment_id=job_compartment_id,
description=job_description,
model_details=job_model_details,
input_location=job_input_location,
output_location=job_output_location)

In [ ]:
#call the AI Create Transcription Job 
transcription_job = None
try:
    transcription_job = ai_client.create_transcription_job(create_transcription_job_details=transcription_job_details)
except Exception as e:
    print(e)
else:
    print(transcription_job.data)

In [ ]:
#update name and description of newly created job
UPDATE_JOB_DETAILS = oci.ai_speech.models.UpdateTranscriptionJobDetails(display_name=new_display_name, description=new_description)

try:
    transcription_job = ai_client.update_transcription_job(transcription_job.data.id, UPDATE_JOB_DETAILS)
    print(transcription_job.data)
except Exception as e:
    print(e)

In [ ]:
# Move Job to new compartment (LiveLab)
COMPARTMENT_DETAILS = oci.ai_speech.models.ChangeTranscriptionJobCompartmentDetails(compartment_id=new_compartment_id)

try:
    ai_client.change_transcription_job_compartment(transcription_job.data.id, COMPARTMENT_DETAILS)
except Exception as e:
    print(e)

In [ ]:
# Gets All Transcription Jobs from a particular compartment (LiveLab)
try:
    transcription_jobs = ai_client.list_transcription_jobs(compartment_id=transcription_job.data.compartment_id)
except Exception as e:
    print(e)
else:
    print(transcription_jobs.data)

In [ ]:
# Gets Transcription Job with given Transcription job id
try:
    if transcription_job.data:
        transcription_job = ai_client.get_transcription_job(transcription_job.data.id)
except Exception as e:
    print(e)
else:
    print(transcription_job.data)

In [ ]:
# Cancel Transcription Job
try:
    ai_client.cancel_transcription_job(transcription_job.data.id)
except Exception as e:
    print(e)

In [ ]:
# Gets Transcription tasks under given transcription Job Id
transcription_tasks = None
try:
    transcription_tasks = ai_client.list_transcription_tasks(transcription_job.data.id, limit=1)
except Exception as e:
    print(e)
else:
    print(transcription_tasks.data)

In [ ]:
# Gets a Transcription Task with given Transcription task id under Transcription Job id
transcription_task = None
try:
    if transcription_tasks.data:
        transcription_task = ai_client.get_transcription_task(transcription_job.data.id, transcription_tasks.data.items[0].id)
except Exception as e:
    print(e)
else:
    print(transcription_task.data)

In [ ]:
# Cancel Transcription Task
try:
    if transcription_task:
        ai_client.cancel_transcription_task(transcription_job.data.id, transcription_task.data.id)
except Exception as e:
    print(e)